## Imswitch/UC2-REST DEMO for 8P370 CBL Microscopy

**By Marcus Vroemen and Tom van Hattem**

*Using the open UC2 Rest and Imswitch Software*

This tutorial and demo will shows you how to get started with Imswitch and UC2 in Python. Imswitch is a software package that allows the creation of graphical user interfaces (GUIs) and interact with our hardware, the open UC2 Controller Board to turn on LED matrices or control the motors. The board itself will not handle cameras, these will usually be handled over USB. Most online tutorials will show you. how to flash the boards. This should already be done on the controller boards, but if not they can easily be reflashed with help of this page: https://youseetoo.github.io/ <br>
But before reflashing your computer will require drivers to see the controller board. If the official openUC2 boards are used with a ESP32-WROOM-32D controller and shield, you will need the CP210x drivers: https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers?tab=downloads
It is recommended to use Visual Studio Code or PyCharm, although Spyder is also possible (but please don't). <br>

IMPORTANT: Do not attempt to use the motors without an external power supply. Powering over USB will most certainly not work/damage your computer.

*Sidenote: also ensure that you are using a data cable. Charging micro usb cables will not be able to interact with the controller board. <br>*

Let's get started. First we need to configure the python environment


### Python Environment Configuration




1. Create a new Python environment called `8P370`. Do this within the Anaconda command prompt, run:


```python
conda create --name 8P370 python=3.8
```

2. Make sure to activate the `8P370` environment before proceeding to install pacakges with the following command:


```python
conda activate 8P370
```



3. Next, install `UC2-REST` and `ImSwitch`  packages (and other packages you might need at the place of <...>) with `pip` using the following command:

```python
python3 -m pip install -U pip
pip install UC2-REST --user
pip install ImSwitchUC2==0.2.0.14
pip install ipykernel 
pip install opencv-python numpy [Note: not required but recommeded]
pip install <...>             
```

Hint: Make sure to only use `pip` from now on. Do not mix `pip` with `conda install`. 

Or use the command 'conda env create -f 8P370_v2.yml' (or v3) if you cloned the current github repository to directly install the environment. Make sure your terminal is in the correct folder or use cd path\to\repo to go to the correct folder.

###  Basic Python Code to control the microscope

To use python code to control the microscope, we need to communicate with the UC2 Controller board. This is a USB connection through COM ports. In Windows open the device manager, under Ports(COM&LPT) when the board is connected it should show `Silicon Labs CP210x USB to UART Bridge (COM4)`. COM4 could be a different port but by default it usually is 3 or 4. Change in the next code block the serialport to your COM port. It has to be a string.

In [ ]:
%reload_ext autoreload 
%autoreload 2

#Import necessary libraries
import uc2rest as uc2
import numpy as np 
import time

In [ ]:
"""Open communication with the UC2 Controller Board
#The log should print '{identifier_name:... etc.}'. If you restart the kernel please rerun this script
to ensure a com port connection has been established and is available to use. If only the state is printed 
it is likely that you have already run this code and the connection is still available in the kernel.
"""

serialport = "COM5" # for Windows - change accordingly
#serialport = "/dev/cu.SLAB_USBtoUART" # for MAC change accordingly
#serialport = "/dev/cu.wchusbserial110" # for MAC change accordingly

if 'ESP32' not in locals():
    ESP32 = uc2.UC2Client(serialport=serialport)
_state = ESP32.state.get_state()
print(_state)

# LED MATRIX

Now that we have initialized the COM port and UC2 controller we can test the LED matrix. Connect it to the LED1 port.

<img src="img//IMG20230926153841.jpg" alt="alt text" width="500"/>

In [ ]:
# test LED
print("The LED pin is: "+str(ESP32.led.get_ledpin()))
time.sleep(2)
ESP32.led.send_LEDMatrix_full(intensity=(255, 255, 255))
time.sleep(2.5)
ESP32.led.send_LEDMatrix_full(intensity=(0, 0, 0))

# Initializing and Moving the motors
Now lets connect the z-stage. Make sure to connect the external power supply, please don't blow up your usb ports. 

The following code snippets will help you moving the motors (XYZ) continously or at a known number of steps at a certain speed level (all measured in steps/s). <br>
**In general:** The axes are: 
A => 0
X => 1
Y => 2
Z => 3

The pin configuration for the uc2 board can be found at: https://youseetoo.github.io/

For the Z-stage:

<img src="img//IMG20230926161544.jpg" alt="alt text" width="350"/>

In [ ]:
# we don't want to change the configuration now
# OR setup motors individually (according to WEMOS R32 D1)
if 0:
    ESP32.motor.set_motor(stepperid = 1, position = 0, stepPin = 26, dirPin=16, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 2, position = 0, stepPin = 25, dirPin=27, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 3, position = 0, stepPin = 17, dirPin=14, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 0, position = 0, stepPin = 19, dirPin=18, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)


#ESP32.motor.set_motor(stepperid = 1, position = 0, stepPin = 2, dirPin=33, enablePin=13, maxPos=None, minPos=None, acceleration=None, isEnable=1)
#ESP32.motor.set_motor(stepperid = 0, position = 0, stepPin = 22, dirPin=21, enablePin=13, maxPos=None, minPos=None, acceleration=None, isEnable=1)
#ESP32.motor.set_motor(stepperid = 3, position = 0, stepPin = 12, dirPin=14, enablePin=13, maxPos=None, minPos=None, acceleration=None, isEnable=1)

This pin configuration also can be reset to default values. For more information on the build in functions see this UC2-REST notebook: https://github.com/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client-PinConfigurator.ipynb



In [ ]:
position1 = ESP32.motor.get_position(timeout=1)
print(position1)

ESP32.motor.move_z(steps=-10000, speed=5000, is_blocking=True)
ESP32.motor.move_z(steps=10000, speed=5000, is_blocking=True)

ESP32.motor.move_z(steps=-10000, speed=10000, is_blocking=True)
ESP32.motor.move_z(steps=10000, speed=10000, is_blocking=True)

ESP32.motor.move_z(steps=-10000, speed=100000, is_blocking=True)
ESP32.motor.move_z(steps=10000, speed=100000, is_blocking=True)

time.sleep(1) 

position2 = ESP32.motor.get_position(timeout=1)
print(position2)

The ESP32 object will keep an connection to the board open until you reset the kernel or disconnect the USB. To close the connection with the software please use the command below. 

In [ ]:
ESP32.close()

A more extensive tutorial on the different features can be found in the official UC2-REST git: https://github.com/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client.ipynb

# IMSWITCH

So far we have only used UC2-REST, but we can also use IMSWITCH. IMSWITCH provides a nice user interface. It is easiest to install a new environment. You can follow the commands below in anaconda cmd or in a build in environment (but the conda should be in PATH). The commands first clone the openuc2 fork of imswitch, then install a new environment based on the requirements from the fork, then downloads config files from Ben (one of the creators of openUC2). These config files should be located in your documents folder. 


```python
cd %HOMEPATH%\Documents
git clone https://github.com/openUC2/ImSwitch/
cd ImSwitch

conda create -n imswitch python=3.9 -y
conda activate imswitch
pip install -r requirements.txt --user

cd %HOMEPATH%\Documents
```


Note: if there is a folder called ImSwitchConfig => rename it!

```python	
git clone https://github.com/beniroquai/ImSwitchConfig
```

To start imswitch use this command in the cmd line.
```python
cd %HOMEPATH%\Documents\ImSwitch
python3 imswitch
```

Work In Progress, more documentation coming soon.

# Manual Camera Control
Camera control can be achieved within python and with external software. Different setups require different cameras and software needs. For most automation tasks it is likely needed to access the camera within python. For other programs it might be easier to access through an external application.

### Alied Vision 1800 U-158M Camera
The Alied Vision camera is meant for fluorescence detection due to high sensitivity. The software used for the Alied Vision Camera is VimbaX. The software [`VimbaX_Setup-2023-1-Win64.exe`] can be found on this page: https://www.alliedvision.com/en/products/software/vimba-x-sdk/ Make sure to connect the camera to your laptop during installation. This installation includes the vimba X viewer, which you can use to view the camera directly as a standalone program. Within the viewer use Camera>Freerun command to activate the camera (or use CTRL+F).

<img src="img/AV-cam.jpg" alt="Alied Vision Camera" style="width: 250px; margin-right: 20px;">

To use the camera in python we need a few more steps. This SDK by default installs in C:\Program Files. Within the installation we need the path to the python wheel file which can be found here: `C:\Program Files\Allied Vision\Vimba X\api\python`. Within the anaconda prompt (or another cmd with python envs active) we change the directory to the Vimba Install directory and install from the .whl file with the following commands:

```python
cd "C:\\Program Files\\Allied Vision\\Vimba X\\api\\python"
pip install vmbpy-1.0.2-py3-none-any.whl
```

Further documentation can be found at: https://docs.alliedvision.com/Vimba_X/Vimba_X_DeveloperGuide/pythonAPIManual.html

In [ ]:
#Test to see if the camera is now available. 
from vmbpy import *
with VmbSystem.get_instance () as vmb:
    cams = vmb.get_all_cameras ()

print(cams)

In [ ]:
import cv2
from vmbpy import *

with VmbSystem.get_instance () as vmb:
    cams = vmb.get_all_cameras ()
    print(cams)
    print(cams[0])

    with cams[0] as cam:
        frame = cam.get_frame ()
        frame.convert_pixel_format(PixelFormat.Mono8)
        cv2.imwrite('frame.jpg', frame.as_opencv_image ())


### ArduCam
The easiest way to access the camera is through the build-in windows camera app. You will have to switch cameras as the default it your laptops webcam. <br>

The following code shows how to capture and save an image. 
ArduCam Spec Sheet: https://www.arducam.com/product/b0196arducam-8mp-1080p-usb-camera-module-1-4-cmos-imx219-mini-uvc-usb2-0-webcam-board-with-1-64ft-0-5m-usb-cable-for-windows-linux-android-and-mac-os/

<img src="img/arducam.jpg" alt="Alied Vision Camera" style="width: 250px; margin-right: 20px;">

#### Commands to install the OpenCV Library
```python
pip install opencv-python
```

In [ ]:
# import the opencv library 
import cv2 

# define a video capture object 
vid = cv2.VideoCapture(0) 

#Filepath where image is written
file_path = "ArduCam\\"

while(True): 

    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 

    # Display the resulting frame 
    cv2.imshow('frame', frame) 

    #cv2.imwrite(file_path, frame)

    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows()

# Delta Stage
The delta stage is able to move the sample in 3 dimensions. When connecing the stage to the UC2 controller board, make sure all connections are facing the same direction (marked with tape), the board contains three motor driver boards and the cables are free from the gears. When using the stage, first try to move each motor so that each side is set at the same height and make sure to not go too far from this position as it might break the stage!

In [43]:
# Basic demonstration on how to move the motors
Z_change_up =    10000
Z_change_down = -10000
speed =          10000

ESP32.motor.move_xyz(
    steps=(Z_change_up,Z_change_up,Z_change_up), 
    speed=(speed,speed,speed), 
    acceleration=None, 
    is_blocking=False, 
    is_absolute=False, 
    is_enabled=True
    )

time.sleep(1) 

ESP32.motor.move_xyz(
    steps=(Z_change_down,Z_change_down,Z_change_down), 
    speed=(speed,speed,speed), 
    acceleration=None, 
    is_blocking=False, 
    is_absolute=False, 
    is_enabled=True
    )

ClearCommError failed (PermissionError(13, 'Access is denied.', None, 5))


### Convert coordinates
The motorised delta requires a coordinate system transform. This is explained in the following openflexure forum thread:
https://openflexure.discourse.group/t/delta-stage-geometry/628/2

In [ ]:
# Code to convert cartesian to desta stage coordinates
def convert_coordinates(x, y, z, camera_angle=0):
    """
    Transform Cartesian coordinates to input values for a delta stage.

    Parameters:
    - x (float): X-coordinate.
    - y (float): Y-coordinate.
    - z (float): Z-coordinate.
    - camera_angle (float, optional): Angle of camera rotation relative to the stage in degrees (default is 0).

    Returns:
    - Tuple[int, int, int]: Transformed delta stage coordinates rounded to the nearest integer.

    This function takes Cartesian coordinates (x, y, z) and converts them to input values
    suitable for a delta stage. It incorporates a camera rotation relative to the stage
    specified by the `camera_angle`. The resulting coordinates are rounded to the nearest integer.
    """

    # Make array from cartesian coordinates
    cartesian_coordinates = np.array([x, y, z])

    # Setup parameters for conversion (might need adjustment)
    flex_h = 70
    flex_a = 35
    flex_b = 47

    # Set up camera rotation relative to stage
    camera_theta = (camera_angle / 180) * np.pi
    R_camera = np.array([
        [np.cos(camera_theta), -np.sin(camera_theta), 0],
        [np.sin(camera_theta), -np.cos(camera_theta), 0],
        [0, 0, 1]
    ])

    # Transformation matrix converting delta into cartesian
    x_fac = -1 * np.multiply(np.divide(2, np.sqrt(3)),
                             np.divide(flex_b, flex_h))
    y_fac = -1 * np.divide(flex_b, flex_h)
    z_fac = np.multiply(np.divide(1, 3), np.divide(flex_b, flex_a))

    Tvd = np.array([
        [-x_fac, x_fac, 0],
        [0.5 * y_fac, 0.5 * y_fac, -y_fac],
        [z_fac, z_fac, z_fac]
    ])

    # Transform coordinates
    delta_coordinates = np.linalg.inv(Tvd) @ R_camera @ cartesian_coordinates
    delta_coordinates = np.round(delta_coordinates).astype(int)

    # print("Delta Coordinates:", delta_coordinates)

    return delta_coordinates[0], delta_coordinates[1], delta_coordinates[2]


delta_coordinates = convert_coordinates(x=0, y=10000, z=0, camera_angle=0)

ESP32.motor.move_xyz(
    steps=(delta_coordinates[0], delta_coordinates[1], delta_coordinates[2]),
    speed=(10000, 10000, 10000),
    acceleration=None,
    is_blocking=False,
    is_absolute=False,
    is_enabled=True
)

# TROUBLESHOOTING
Once the `ESP32 = uc2.UC2Client(serialport=serialport)` command is executed, within the current kernel, the ESP32 class is available. To close this the port needs to be closed with .close(). If you reattept to run the command again, it will provide a error such as: 
```python
PermissionError(13, 'Access is denied.', None, 5)
``` 
As two instances are not possible. Please restart the kernel to proceed. If anything else goes wrong, also just restart the kernel. For the permission error, it is also likely that another program or kernel is running and has the com port opened. For example a program such as Ultimaker CURA or Arduino IDE is likely to interfere with UC2, therefore close it and disable opening the program on startup in the task manager (for convenience). 

```python
FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
``` 
For a not found error it is likely that the COM port is wrong (or usb not connected properly). 

There are several things that limit the board from being connected correctly to your laptop/pc. The first thing to check is if you have the correct drivers installed: https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers?tab=downloads

Next check if you are using the correct cable, and if it shows up in the device manager. 



Also check that you are using the correct com port (as seen in the device manager.)

To check if the hardware works correctly it is also possible to check using https://youseetoo.github.io/indexWebSerialTest.html. This will ask in the browser to connect to the UC2 controller board and if a Z-stage motor or LED ring is correctly connected to the right port (labelled on the pcb) then you should be able to control them. 

Sometimes `pip UC2-REST --force-reinstall --user` will cause the code to execute properly. Feel free to try this. Remember to first activate your env 8P370 in anaconda (or other env manager).


In [ ]:
#Try to manually open the serial port to check if this works. 
# import serial
# import time
# c = serial.Serial('COM76', 9600)
# counter = 0
# while True:
#     signal = c.read()
#     print("running")
#     print(signal)
#     time.sleep(0.1)
#     c.flushOutput()
#     counter +=1
#     if counter == 10:
#         c.close()
#         break